In [1]:
# Hacemos un try except, para que si ya existe el recurso instalado, simplemente lo cargue y no lo instalemos sí o sí
try:
  from google.colab import auth
  auth.authenticate_user()
  import pandas as pd
  from google.cloud import storage
except:
  !pip install google-cloud-storage
  !pip install pandas

In [5]:
#
# Estas funciones las podemos implementar en cada uno de nuestros notebooks
# para hacer uso de un CSV y crear otro CSV, y todo ello contra los buckets
# creados en el Google Cloud Storage de la arquitectura de la práctica
#
# Es muy importante tener permisos de Storage Object Creator, si no te deja
# hacer nada, pide permiso al administrador
#
# El bucket de los csv de "origen" o no tratados será ds-edw-raw-d1655b14
# El bucket de los csv "finales" o procesados será ds-edw-processed-d1655b14
#


# Descargamos el CSV con el que vamos a trabajar
def descargar_de_bucket_origen(nombre_bucket_origen, nombre_blob_origen, nombre_archivo_destino):
    cliente = storage.Client()
    bucket = cliente.bucket(nombre_bucket_origen)
    blob = bucket.blob(nombre_blob_origen)
    blob.download_to_filename(nombre_archivo_destino)
    print(f"Archivo {nombre_blob_origen} descargado como {nombre_archivo_destino}.")

nombre_bucket_origen = 'ds-edw-raw-d1655b14'
nombre_bucket_procesado = 'ds-edw-processed-d1655b14'
nombre_blob_origen = 'dataset/hascosva_2022.csv' # Aquí se pone el nombre de la carpeta CSV, barra / , y el nombre del csv con el que queremos trabajar
ruta_descarga_local = 'hascosva_2022.csv' # Aquí se pone solamente el nombre del csv con el que queremos trabajar
descargar_de_bucket_origen(nombre_bucket_origen, nombre_blob_origen, ruta_descarga_local)

# Trabajamos con el CSV y hacemos lo que haya que hacer con la fuente
df = pd.read_csv(ruta_descarga_local)

# Aquí haríamos los cambios que fuesen necesarios
#  CODIGO
#  CODIGO , etc

# Una vez trabajado lo anterior, creamos nombre del fichero de destino y generamos dicho CSV
csv_tratado = 'PonerCualquierNombre.csv'  # Acordaos de cambiar el nombre por el que corresponda
df.to_csv(csv_tratado, index=False)

# Subimos los cambios o tratamientos a un nuevo  bucket de procesados con un CSV nuevo
def subir_a_bucket_destino(nombre_bucket_procesado, nombre_archivo_destino, nombre_blob_destino):
    cliente = storage.Client()
    bucket = cliente.bucket(nombre_bucket_procesado)
    blob = bucket.blob(nombre_blob_destino)
    # Verificar si el archivo ya existe en el bucket
    if blob.exists():
        print(f"El archivo {nombre_blob_destino} ya existe en el bucket {nombre_bucket_procesado} y no se ha subido por no sobreescribirlo y preservar los datos, pon otro nombre o solicita ayuda al administrador del bucket.")
    else:
        blob.upload_from_filename(nombre_archivo_destino)
        print(f"Archivo {nombre_archivo_destino} subido a {nombre_blob_destino} en el bucket {nombre_bucket_procesado}.")

nombre_blob_destino = 'CSV/PonerCualquierNombre.csv' # Acordaos de cambiar el nombre por el que corresponda
subir_a_bucket_destino(nombre_bucket_procesado, csv_tratado, nombre_blob_destino)


Archivo dataset/hascosva_2022.csv descargado como hascosva_2022.csv.
El archivo CSV/PonerCualquierNombre.csv ya existe en el bucket ds-edw-processed-d1655b14 y no se ha subido por no sobreescribirlo y preservar los datos, pon otro nombre o solicita ayuda al administrador del bucket.
